# Detection Notebook

### Load Log Data


In [ ]:
%load_ext autoreload
%autoreload 2

import os
import glob


import pandas as pd
import numpy as np
import plotly.express as px


path = "/home/patrick/github/fibsem/fibsem/log/data"
df = pd.read_csv(os.path.join(path, "data.csv"))


In [ ]:
# plot p.x, p.y color by feature
fig = px.scatter(df, x="p.x", y="p.y", color="feature")
fig.show()


In [ ]:

from fibsem.detection import detection
import random
# get list of unique images
filenames = df["image"].unique()
random.shuffle(filenames)

i=0

# loop through each image, get all features
for fname in filenames:

    det = detection._det_from_df(df, path, fname)

    if det is None:
        continue

    detection.plot_det_result_v2(det)

    if i==20:
        break

    i+=1



## Multiple Feature Detection

P1: Contour Detection

P2: Instance Segmentation Model



In [ ]:
%load_ext autoreload
%autoreload 2

import os
from copy import deepcopy
from pathlib import Path

import napari
import napari.utils.notifications
import numpy as np
import tifffile as tff
from PyQt5 import QtWidgets

from fibsem.detection import detection
from fibsem.detection import utils as det_utils
from fibsem.detection.detection import DetectedFeatures
from fibsem.segmentation import model as fibsem_model
from fibsem.segmentation.model import load_model
from fibsem.structures import (
    BeamType,
    FibsemImage,
    Point,
)
from PyQt5.QtCore import pyqtSignal
from fibsem.ui.qtdesigner_files import FibsemDetectionWidget
import logging

CHECKPOINT_PATH = os.path.join(os.path.dirname(fibsem_model.__file__), "models", "model4.pt")

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from copy import deepcopy
from pathlib import Path

import napari
import napari.utils.notifications
import numpy as np
import tifffile as tff
from PyQt5 import QtWidgets

from fibsem.detection import detection
from fibsem.detection import utils as det_utils
from fibsem.detection.detection import DetectedFeatures
from fibsem.segmentation import model as fibsem_model
from fibsem.segmentation.model import load_model
from fibsem.structures import (
    BeamType,
    FibsemImage,
    Point,
)
from PyQt5.QtCore import pyqtSignal
from fibsem.ui.qtdesigner_files import FibsemDetectionWidget
import logging

from fibsem.detection.detection import Feature, LamellaCentre, LamellaLeftEdge, LamellaRightEdge
import matplotlib.pyplot as plt

CHECKPOINT_PATH = os.path.join(os.path.dirname(fibsem_model.__file__), "models", "model4.pt")

In [ ]:
checkpoint = str(CHECKPOINT_PATH)
encoder="resnet34"
num_classes = 3
model = load_model(checkpoint=checkpoint, encoder=encoder, nc=num_classes)

features = [detection.NeedleTip(), detection.LamellaCentre()]

image = FibsemImage.load(os.path.join(os.path.dirname(detection.__file__), "test_image.tif"))

In [ ]:
# detect features
features = [detection.NeedleTip(), detection.LamellaCentre(), 
            detection.LamellaLeftEdge(), detection.LamellaRightEdge(), 
            detection.LandingPost(), detection.ImageCentre()]
pixelsize = image.metadata.pixel_size.x if image.metadata is not None else 25e-9
det = detection.locate_shift_between_features_v2(
    deepcopy(image.data), model, features=features, pixelsize=pixelsize
)

# plot image and mask overlay
detection.plot_det_result_v2(det)

In [ ]:
det.distance

In [ ]:


def create_random_image(n: int = 3):
    image = np.zeros(shape=(1024, 1536 ))
    # create two squares at two random non-overlapping locations
    x1 = np.random.randint(0, image.shape[1])
    y1 = np.random.randint(0, image.shape[0])

    x2 = np.random.randint(0, image.shape[1])
    y2 = np.random.randint(0, image.shape[0])

    while (x1-x2)**2 + (y1-y2)**2 < 100**2:
        x2 = np.random.randint(0, image.shape[1])
        y2 = np.random.randint(0, image.shape[0])

    image[y1:y1+100, x1:x1+100] = 1
    image[y2:y2+100, x2:x2+100] = 1

    # convert image to int
    image = image.astype(np.uint8)
    return image

img1 = create_random_image()
img2 = create_random_image()
img3 = create_random_image()
image = img1 + img2 + img3
mask = np.clip(image, image, 1)


In [ ]:
features = [
            detection.LamellaCentre(), 
            # detection.LamellaLeftEdge(), 
            # detection.LamellaRightEdge(), 
            ]


point = Point(1400, 800)
features = detection.detect_features_v2(image, mask, features, filter=True, point=None)

det = DetectedFeatures(features, image, mask, det_utils.decode_segmap(mask), pixelsize=25e-9)

detection.plot_det_result_v2(det)

from pprint import pprint 
pprint(features)

features = [
            detection.LamellaCentre(), 
            # detection.LamellaLeftEdge(), 
            # detection.LamellaRightEdge(), 
            ]

features = detection.detect_features_v2(image, mask, features, filter=False)

det = DetectedFeatures(features, image, mask, det_utils.decode_segmap(mask), pixelsize=25e-9)

detection.plot_det_result_v2(det)

from pprint import pprint 
pprint(features)